## Bot Vs Human Classification using L layer Feed Forward Network

### Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import sys
import os

# Get the absolute path to the Shared folder
shared_path = os.path.abspath('..')

# Add to sys.path if not already there
if shared_path not in sys.path:
    sys.path.append(shared_path)
print(sys.path)

plt.style.use('ggplot')

['/home/sriram/stacksville/repos/cari-seetha', '/home/sriram/stacksville/repos/cari-seetha/api', '/home/sriram/stacksville/repos/tcup-seetha', '/home/sriram/stacksville/repos/tcup-seetha/tcup', '/home/sriram/projects/deep_learning/user_vs_bot', '/usr/lib64/python312.zip', '/usr/lib64/python3.12', '/usr/lib64/python3.12/lib-dynload', '', '/home/sriram/projects/deep_learning/venv/lib64/python3.12/site-packages', '/home/sriram/projects/deep_learning/venv/lib/python3.12/site-packages', '/home/sriram/projects/deep_learning']


### Load Dataset

In [2]:
df = pd.read_csv('bots_vs_users.csv')
df.head()

,has_domain,has_birth_date,has_photo,can_post_on_wall,can_send_message,has_website,gender,has_short_name,has_first_name,has_last_name,...,ads_ratio,avg_views,posting_frequency_days,phone_numbers_ratio,avg_text_uniqueness,city,has_occupation,occupation_type_university,occupation_type_work,has_personal_data
0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown
1,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown
2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown
3,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown


### Data Cleaning

In [3]:
## Checking for NaNs

cols = df.isna().sum()
cols_with_nans = cols[cols > 0]
cols_with_nans

posts_count               4483
avg_likes                 4483
links_ratio               4483
hashtags_ratio            4483
avg_keywords              4483
avg_text_length           4483
attachments_ratio         4483
avg_comments              4483
reposts_ratio             4483
ads_ratio                 4483
avg_views                 4483
posting_frequency_days    4483
phone_numbers_ratio       4483
avg_text_uniqueness       4483
dtype: int64

In [4]:
### Replacing NaN with 0
df.fillna(0, inplace=True)

In [5]:
df[cols_with_nans.index].describe()

,posts_count,avg_likes,links_ratio,hashtags_ratio,avg_keywords,avg_text_length,attachments_ratio,avg_comments,reposts_ratio,ads_ratio,avg_views,posting_frequency_days,phone_numbers_ratio,avg_text_uniqueness
count,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000
mean,4.411474,4.630152,0.037051,0.034081,1.830669,129.000695,0.212165,0.144644,0.102688,0.000632,147.727986,15.707375,0.000134,0.158189
std,8.183516,22.038715,0.118964,0.109672,6.396108,430.827974,0.389802,1.225439,0.267299,0.016160,830.462351,80.493815,0.004650,0.305224
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,20.000000,894.900000,1.000000,1.000000,61.600000,5841.250000,1.000000,64.050000,1.000000,0.950000,41415.700000,2866.590000,0.300000,1.000000


In [6]:
df.shape

(5874, 60)

In [7]:
df.columns

Index(['has_domain', 'has_birth_date', 'has_photo', 'can_post_on_wall',
       'can_send_message', 'has_website', 'gender', 'has_short_name',
       'has_first_name', 'has_last_name', 'access_to_closed_profile',
       'is_profile_closed', 'target', 'has_nickname', 'has_maiden_name',
       'has_mobile', 'all_posts_visible', 'audio_available', 'has_interests',
       'has_books', 'has_tv', 'has_quotes', 'has_about', 'has_games',
       'has_movies', 'has_activities', 'has_music', 'can_add_as_friend',
       'can_invite_to_group', 'subscribers_count', 'is_blacklisted',
       'has_career', 'has_military_service', 'has_hometown', 'marital_status',
       'has_universities', 'has_schools', 'has_relatives', 'is_verified',
       'is_confirmed', 'has_status', 'posts_count', 'avg_likes', 'links_ratio',
       'hashtags_ratio', 'avg_keywords', 'avg_text_length',
       'attachments_ratio', 'avg_comments', 'reposts_ratio', 'ads_ratio',
       'avg_views', 'posting_frequency_days', 'phone_numbe

In [8]:
df['gender'].value_counts()

gender
1.0        4512
2.0        1306
0.0          32
Unknown      24
Name: count, dtype: int64

In [9]:
## Dropping city

df=df.drop(columns=['city']).copy()

In [10]:
df.replace('Unknown', -1, inplace=True)
df.head()

,has_domain,has_birth_date,has_photo,can_post_on_wall,can_send_message,has_website,gender,has_short_name,has_first_name,has_last_name,...,reposts_ratio,ads_ratio,avg_views,posting_frequency_days,phone_numbers_ratio,avg_text_uniqueness,has_occupation,occupation_type_university,occupation_type_work,has_personal_data
0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,-1,-1
1,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,-1,-1
2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,-1,-1
3,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,-1,-1
4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,-1,-1


In [11]:
df.describe()

,target,posts_count,avg_likes,links_ratio,hashtags_ratio,avg_keywords,avg_text_length,attachments_ratio,avg_comments,reposts_ratio,ads_ratio,avg_views,posting_frequency_days,phone_numbers_ratio,avg_text_uniqueness
count,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000,5874.000000
mean,0.500000,4.411474,4.630152,0.037051,0.034081,1.830669,129.000695,0.212165,0.144644,0.102688,0.000632,147.727986,15.707375,0.000134,0.158189
std,0.500043,8.183516,22.038715,0.118964,0.109672,6.396108,430.827974,0.389802,1.225439,0.267299,0.016160,830.462351,80.493815,0.004650,0.305224
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,20.000000,894.900000,1.000000,1.000000,61.600000,5841.250000,1.000000,64.050000,1.000000,0.950000,41415.700000,2866.590000,0.300000,1.000000


In [12]:
## Checking target

df['target'].value_counts()

target
1    2937
0    2937
Name: count, dtype: int64

### Dataset Preparation

In [13]:
dataset = df.drop(['target'], axis=1).to_numpy()
target = df['target'].to_numpy().reshape(-1, 1)

(dataset.shape, target.shape)

((5874, 58), (5874, 1))

In [14]:
from core.normalizers import min_max_scaler

dataset = min_max_scaler(dataset)
dataset.shape

(5874, 58)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(dataset, target, test_size=0.25, random_state=42, stratify=target)

X_train = X_train.T
X_test = X_test.T
Y_train = Y_train.T
Y_test = Y_test.T
print(f'Train -> X - {X_train.shape}, Y - {Y_train.shape}')
print(f'Test -> X - {X_test.shape}, Y - {Y_test.shape}')

Train -> X - (58, 4405), Y - (1, 4405)
Test -> X - (58, 1469), Y - (1, 1469)


In [16]:
np.unique(Y_test, return_counts = True)

(array([0, 1]), array([735, 734]))

## Training Neural Network

In [17]:
from core.fnn import FeedForwardNetwork

L = 4
layer_dims = [X_train.shape[0], 128, 64, 32, 1]
activations = ['relu', 'relu', 'relu', 'sigmoid']
nn = FeedForwardNetwork(L, layer_dims, activations)
nn.info()

Layers:  4
Layer Dims:  [58, 128, 64, 32, 1]


In [18]:
nn.train(X_train, Y_train, learning_rate=0.5, iterations=2500)

cost after 0 iterations:  0.6931475076240259
cost after 100 iterations:  0.6931427956822848
cost after 200 iterations:  0.6931382173828213
cost after 300 iterations:  0.6931276285913369
cost after 400 iterations:  0.6930933563674847
cost after 500 iterations:  0.6928603673353586
cost after 600 iterations:  0.26864452085285245
cost after 700 iterations:  0.1408339079018908
cost after 800 iterations:  0.12805332493958335
cost after 900 iterations:  0.11582040569997375
cost after 1000 iterations:  0.12000539450783347
cost after 1100 iterations:  0.11552330474162532
cost after 1200 iterations:  0.10428797526619049
cost after 1300 iterations:  0.10285745580662761
cost after 1400 iterations:  0.09872294164787179
cost after 1500 iterations:  0.09768129049356969
cost after 1600 iterations:  0.09623532081665098
cost after 1700 iterations:  0.10239210877707038
cost after 1800 iterations:  0.09323999559033924
cost after 1900 iterations:  0.09518442560973844
cost after 2000 iterations:  0.09314291

np.float64(0.0890186792942627)

In [21]:
output = nn.predict(X_train)
predicted = np.where(output > 0.5, 1, 0)
accuracy = np.mean(predicted == Y_train)
print('Accuracy (Fit): ', accuracy * 100, '%')

Accuracy (Fit):  97.27582292849036 %


In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.squeeze(predicted), np.squeeze(Y_train))

array([[2148,   66],
       [  54, 2137]])

#### Validation

In [23]:
output = nn.predict(X_test)
predicted = np.where(output > 0.5, 1, 0)
accuracy = np.mean(predicted == Y_test)
print('Accuracy (Fit): ', accuracy * 100, '%')

Accuracy (Fit):  96.73247106875425 %


In [24]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.squeeze(predicted), np.squeeze(Y_test))

array([[716,  29],
       [ 19, 705]])